In [ ]:
""" 
# ALURA - Desafio: 7DaysOfCode - Junho/2024
# Processo de importação e tratamento dos dados (dias 1 e 2)
"""
import pandas as pd
import os
#
pd.options.display.max_rows = 10000
pd.options.display.max_columns = 30
#
_dicNomeDF        = dict()     # Dicionário utilizado para unificar os arquivos CSV
_intContReg       = 0          # Contador dos registros
_lstNome_CDU      = []         # Lista que armazena temporariamente o nome da CDU
_lstDataFrames    = []         # Lista que contém o(s) DataFrame(s) temporariamente
_strExtArquivo    = '.csv'
_strDB_CDU        = 'DB_CDU' + _strExtArquivo
_strDB_Emprestimo = 'DB_Emprestimo' + _strExtArquivo
_strDB_CadLivros  = 'DB_CadLivros' + _strExtArquivo
_strPathData      = r'D:\Users\rtoni\OneDrive\Git-Dados\Projetos\7DaysOfCode.io'       # Pasta que armazena o arquivo
_strURL_GIT       = 'https://raw.githubusercontent.com/RogerioTonini/7_Days_of_Code_Alura-Python-Pandas/main/Dia_1-Importando_dados/Datasets/dados_emprestimos/emprestimos-'

In [ ]:
""" 
# Concatena os arquivos CSV
"""
for _intAno in range(2010, 2021):       # Faixa de Anos que serão analisados: 2010 - 2020    
    for _intQtdArqs in range(1, 3):     # Quantidade de arquivos por Ano = 2
        """
        # _dicNomeDF  - Dicionário que armazena temporariamente os dados dos arquivos
        # _strNomeDF  - Nome do DataFrame atual
        # _strURL_Arq - Endereço da URL do GIT onde estão os dados
        """
        _strNomeDF  = f'df_{str(_intAno)}_{str(_intQtdArqs)}'
        _strURL_Arq = f"{_strURL_GIT}{str(_intAno)}{str(_intQtdArqs)}{_strExtArquivo}?raw=true"
        print(_strNomeDF)
        try:
            """
            # Processo de tratamento dos dados Livros Emprestados:
            # - Converte as colunas que contém Data/Hora para Datatime 
            # - Converte as colunas ['id_emprestimo'] e [matricula_e_siape] para String
            """
            _dicNomeDF[_strNomeDF] = pd.read_csv(_strURL_Arq, sep=",")
            _dicNomeDF[_strNomeDF]['data_emprestimo'] = pd.to_datetime(_dicNomeDF[_strNomeDF]['data_emprestimo'])
            _dicNomeDF[_strNomeDF]['data_devolucao']  = pd.to_datetime(_dicNomeDF[_strNomeDF]['data_devolucao'])
            _dicNomeDF[_strNomeDF]['data_renovacao']  = pd.to_datetime(_dicNomeDF[_strNomeDF]['data_renovacao'])
        except Exception as e:
            print(_strNomeDF, e)
            continue
# Unifica todos os dicionários em um único arquivo. Grava o arquivo em disco
_lstDataFrames = []
[ _lstDataFrames.append(df) for nome_dataframe, df in _dicNomeDF.items() ]

In [ ]:
"""
# Verifica a quantidade total de registros duplicados e os apaga
# Carrega os arquivos df_CadLivros e df_CDU
# Faz o JOIN entre as tabelas df_CadLivros e df_Emprestimo
# Acrescenta a coluna NomeFaixaCDU - Classificação Decimal Universal, conforme código da coluna df_Emprestimo['localizacao']
"""
try:
    _strPathFile  = os.path.join(_strPathData, _strDB_Emprestimo)
    df_Emprestimo = pd.concat(_lstDataFrames, ignore_index=True)     
    df_Emprestimo.to_csv(_strPathFile, header=True, index=False, sep=';')
    df_Emprestimo = pd.read_csv(_strPathFile, sep=';', engine='c')
    print(f'Arquivo {_strDB_Emprestimo} gravado com sucesso!!!')
    #
    _strPathFile = os.path.join(_strPathData, _strDB_CadLivros)
    df_CadLivros = pd.read_parquet('https://github.com/RogerioTonini/7_Days_of_Code_Alura-Python-Pandas/raw/main/Dia_1-Importando_dados/Datasets/dados_exemplares.parquet')
    df_CadLivros.to_csv(_strPathFile, header=True, index=False, sep= ';')
    #
    df_Emprestimo.value_counts()
    df_Emprestimo = df_Emprestimo.drop_duplicates(keep='first')   # Verifica a quantidade total de registros duplicados e os apaga
    df_Emprestimo = df_Emprestimo.merge(df_CadLivros)             # Faz o JOIN entre as tabelas gerando o Dataframe df_Emprestimo_Tratado
    #
    _strPathFile = os.path.join(_strPathData, _strDB_Emprestimo)
    df_Emprestimo.to_csv(_strPathFile, header=True, index=False, sep=';')
    df_Emprestimo = pd.read_csv(_strPathFile, sep=';', engine='c')    
    print(f'Arquivo {_strDB_Emprestimo}: Registros duplicados eliminados e JOIN com Cadastro de Livros realizado com sucesso!!!')
    #
    _strPathFile = os.path.join(_strPathData, _strDB_CDU)
    df_CDU       = pd.read_csv(_strPathFile, sep=';', engine='c')
    #
    for _intCod_CDU in df_Emprestimo['localizacao']:
        #
        for _, _strColuna in df_CDU.iterrows():
            if _intCod_CDU <= _strColuna['Limite_Faixa']:
                _lstNome_CDU.append(_strColuna['Nome_Faixa_CDU'])
                print(f'Cód. Localização: {_intCod_CDU}, Contador: {_intContReg}')
                _intContReg += 1
                break 
    #
    print(f'\n Qtde registros {_strDB_Emprestimo}, {str(_intContReg)}, Qtde lista _lstNome_CDU: {str(len(_lstNome_CDU))} \n')
    df_Emprestimo['Nome_Faixa_CDU']  = _lstNome_CDU
    df_Emprestimo['matricula_siape'] = df_Emprestimo['matricula_ou_siape'].astype(str)
    df_Emprestimo.drop(columns=['matricula_ou_siape', 'localizacao', 'registro_sistema'], inplace = True)  # Exclusão colunas: ['matricula_ou_siape', 'localizacao', 'registro_sistema']
    #
    _strPathFile = os.path.join(_strPathData, _strDB_Emprestimo)
    df_Emprestimo.to_csv(_strPathFile, header=True, index=False, sep=';')
    print('Processo de tratamento/geração do arquivo tratado finalizado!!!')
except OSError as e:
    print(e)